In [1]:
import subprocess
from bs4 import BeautifulSoup
import requests
import os
import time
from tqdm import tqdm
import shlex


In [2]:
contest_id = '1814'
problem_id = 'A'
# time_limit = 1.0

directory = contest_id + '/' + problem_id
replace = {'&quot;': '\"', '&gt;': '>', '&lt;': '<', '&amp;': '&', "&apos;": "'"}
languages = {'GNU C++20 (64)', 'GNU C++14', 'GNU C++11', 'GNU C++17'} # , 'Java 8':'Java 11.0.5', 'Python 3': 'Python 3.7.2', 'Python 2':'Python 2.7.15'

if (not os.path.exists(directory)):
    os.makedirs(directory)
if (not os.path.exists(os.path.join(directory,'compiled'))):
    os.makedirs(os.path.join(directory,'compiled'))

In [3]:
def parse(code):
	for key in replace.keys():
		code = code.replace(key, replace[key])
	return code

#fetches the code corresponding to the parameter url
def getCode(submission_url, submission_id, contest_id):
    source = requests.get(submission_url).text
    soup = BeautifulSoup(source, 'html.parser')
    language = soup.findAll('td')[3].text[6:-6]
    if language not in languages:
        return
    code = soup.findAll('pre')[0].text
    code = parse(code).replace('\r', '')
    if code != '':
        print("Sucessfully fetched", submission_id + ".cpp")
        filewrite = open(directory + '/' + submission_id + '.cpp', 'w')
        filewrite.write(code)
        filewrite.close()

In [5]:
page = 1
max_page = 100
url = "http://codeforces.com/contest/"+contest_id+"/status/"+problem_id+"/page/" + str(page) + "?order=BY_ARRIVED_DESC"
print(url)
for i in tqdm(range(page,max_page+1)):
    url = "http://codeforces.com/contest/"+contest_id+"/status/"+problem_id+"/page/" + str(page) + "?order=BY_ARRIVED_DESC"
    print ("Parsing page:", page, url)
    try:
        source = requests.get(url, timeout=5).text
        soup = BeautifulSoup(source, 'html.parser')
        submissions = soup.findAll('a', {'class':'view-source'})
        
        for submission in submissions:
            submission_url = "http://codeforces.com" + submission['href']
            try:
                getCode(submission_url, submission['submissionid'], contest_id)
            except:
                print("Failed to fetch:", submission_url)
                continue
            time.sleep(1)

    except:
        print("Failed parsing on page:", page)

    page+=1

http://codeforces.com/contest/1814/status/B/page/20?order=BY_ARRIVED_DESC


  0%|          | 0/41 [00:00<?, ?it/s]

Parsing page: 20 http://codeforces.com/contest/1814/status/B/page/20?order=BY_ARRIVED_DESC
Sucessfully fetched 201054737.cpp
Sucessfully fetched 201054686.cpp
Failed to fetch: http://codeforces.com/contest/1814/submission/201054686
Failed to fetch: http://codeforces.com/contest/1814/submission/201054655
Sucessfully fetched 201054615.cpp
Sucessfully fetched 201054590.cpp
Sucessfully fetched 201054589.cpp
Sucessfully fetched 201054563.cpp
Sucessfully fetched 201054551.cpp
Sucessfully fetched 201054488.cpp
Sucessfully fetched 201054487.cpp
Sucessfully fetched 201054485.cpp
Sucessfully fetched 201054484.cpp
Sucessfully fetched 201054471.cpp
Sucessfully fetched 201054470.cpp
Failed to fetch: http://codeforces.com/contest/1814/submission/201054470
Sucessfully fetched 201054440.cpp


  2%|▏         | 1/41 [00:21<14:00, 21.02s/it]

Failed parsing on page: 20
Parsing page: 21 http://codeforces.com/contest/1814/status/B/page/21?order=BY_ARRIVED_DESC
Sucessfully fetched 201054026.cpp
Sucessfully fetched 201054020.cpp
Failed to fetch: http://codeforces.com/contest/1814/submission/201054020
Sucessfully fetched 201054009.cpp
Failed to fetch: http://codeforces.com/contest/1814/submission/201054009
Sucessfully fetched 201054004.cpp
Sucessfully fetched 201053976.cpp
Sucessfully fetched 201053966.cpp
Sucessfully fetched 201053963.cpp
Sucessfully fetched 201053952.cpp
Sucessfully fetched 201053944.cpp
Sucessfully fetched 201053940.cpp


In [4]:
for file in tqdm(os.listdir(directory)):
    file_name = os.path.splitext(file)[0]
    file_ext = os.path.splitext(file)[1]
    try:
        os.system('g++ ' + os.path.join(directory, file_name)+file_ext + ' -o ' + os.path.join(directory,'compiled',file_name) + ' -O2 -DONLINE_JUDGE')    
    except:
        continue

100%|██████████| 809/809 [13:50<00:00,  1.03s/it]


In [6]:
with open('input.txt', 'r') as myfile: 
    test_input = myfile.read()
with open('TESToutput.txt', 'r') as myfile: 
    test_output = myfile.read()

test_output = test_output.lower().rstrip()

time_limit = 2.1

pref = contest_id +'\\'+problem_id
hackable = []
rte = []
tle = []    
for file in tqdm(os.listdir(pref + '\compiled')):
    # print(file)
    cmd = 'more input.txt | ' + pref + '\compiled\\' + file
    try:
        p = subprocess.Popen([pref + '\compiled\\' + file], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=None, text=True)
        p.stdin.write(test_input)
        p.stdin.flush()

        out = None
        try:
            out, err = p.communicate(timeout=time_limit)
        except subprocess.TimeoutExpired:
            p.kill()
            tle.append(os.path.splitext(file)[0])
            continue
            # out, err = p.communicate()
        out = out.strip().lower()
        output_string = ""
        str_list = out.split('Press any key to continue . . .')
        for element in str_list:
            output_string += element
        out = output_string
        # out = p.stdout.readline().strip().lower()
        # print(out)
        # out = subprocess.run(cmd, shell=True, text=True, timeout=time_limit, stderr=None, capture_output=True)
        if (out != test_output):
            hackable.append(os.path.splitext(file)[0])
        pass
            # out = subprocess.run(cmd, shell=True, text=True, timeout=time_limit, stderr=None, capture_output=True).stdout
    except subprocess.TimeoutExpired:
        # print("TLE: ", file)
        tle.append(os.path.splitext(file)[0])
        pass
    except:
        # rte.append(file)
        pass
        
for r in rte:
    print("Ran into runtime exception:", file)

print("Hackable:" )
for h in hackable:
    print('https://codeforces.com/contest/' +contest_id + '/submission/' +  h)

print("")
print("TLE: ")
for t in tle:
    print('https://codeforces.com/contest/' +contest_id + '/submission/' +  t)

100%|██████████| 738/738 [02:39<00:00,  4.63it/s]

Hackable:
https://codeforces.com/contest/1814/submission/200998734
https://codeforces.com/contest/1814/submission/200999414
https://codeforces.com/contest/1814/submission/201004994
https://codeforces.com/contest/1814/submission/201005070
https://codeforces.com/contest/1814/submission/201005371
https://codeforces.com/contest/1814/submission/201005837
https://codeforces.com/contest/1814/submission/201008096
https://codeforces.com/contest/1814/submission/201008861
https://codeforces.com/contest/1814/submission/201013246
https://codeforces.com/contest/1814/submission/201013971
https://codeforces.com/contest/1814/submission/201014078
https://codeforces.com/contest/1814/submission/201018078
https://codeforces.com/contest/1814/submission/201018272
https://codeforces.com/contest/1814/submission/201018528
https://codeforces.com/contest/1814/submission/201018742
https://codeforces.com/contest/1814/submission/201023036
https://codeforces.com/contest/1814/submission/201023249
https://codeforces.co